### In this notbook I train the mouse fullmodels needed for plotting the figures. I will save the models in a checkpoint folder (checkpoints_trained)

I reuse the notbook fullmodel_mouse and fittet it to my purposes

In [1]:
import os
import torch
import numpy as np
from minimodel import data

device = torch.device('cuda')

In [2]:
mouse_id = 4

data_path = './data'
weight_path = './checkpoints_trained'
np.random.seed(1)

In [3]:
# load images
img = data.load_images(data_path, mouse_id, file=data.img_file_name[mouse_id])

raw image shape:  (68000, 66, 264)
cropped image shape:  (68000, 66, 130)
img:  (68000, 66, 130) -2.062947 2.088608 float32


In [4]:
# load neurons
fname = '%s_nat60k_%s.npz'%(data.db[mouse_id]['mname'], data.db[mouse_id]['datexp'])
spks, istim_train, istim_test, xpos, ypos, spks_rep_all = data.load_neurons(file_path = os.path.join(data_path, fname), mouse_id = mouse_id)
n_stim, n_neurons = spks.shape


loading activities from ./data/FX8_nat60k_2023_05_16.npz


In [5]:
# split train and validation set
itrain, ival = data.split_train_val(istim_train, train_frac=0.9)


splitting training and validation set...
itrain:  (32885,)
ival:  (3654,)


In [6]:
# normalize data
spks, spks_rep_all = data.normalize_spks(spks, spks_rep_all, itrain)


normalizing neural data...
finished


In [7]:
ineur = np.arange(0, n_neurons) #np.arange(0, n_neurons, 5)
spks_train = torch.from_numpy(spks[itrain][:,ineur]).to(device)
spks_val = torch.from_numpy(spks[ival][:,ineur]).to(device)

print('spks_train: ', spks_train.shape, spks_train.min(), spks_train.max())
print('spks_val: ', spks_val.shape, spks_val.min(), spks_val.max())

img_train = torch.from_numpy(img[istim_train][itrain]).to(device).unsqueeze(1) # change :130 to 25:100 
img_val = torch.from_numpy(img[istim_train][ival]).to(device).unsqueeze(1)
img_test = torch.from_numpy(img[istim_test]).to(device).unsqueeze(1)

print('img_train: ', img_train.shape, img_train.min(), img_train.max())
print('img_val: ', img_val.shape, img_val.min(), img_val.max())
print('img_test: ', img_test.shape, img_test.min(), img_test.max())

input_Ly, input_Lx = img_train.shape[-2:]

spks_train:  torch.Size([32885, 5804]) tensor(0., device='cuda:0') tensor(69.0541, device='cuda:0')
spks_val:  torch.Size([3654, 5804]) tensor(0., device='cuda:0') tensor(56.4533, device='cuda:0')
img_train:  torch.Size([32885, 1, 66, 130]) tensor(-2.0629, device='cuda:0') tensor(2.0886, device='cuda:0')
img_val:  torch.Size([3654, 1, 66, 130]) tensor(-2.0629, device='cuda:0') tensor(2.0886, device='cuda:0')
img_test:  torch.Size([500, 1, 66, 130]) tensor(-2.0629, device='cuda:0') tensor(2.0886, device='cuda:0')


In [8]:
from minimodel import model_builder
from minimodel import model_trainer
from minimodel import metrics

seed = 1
feve_nlayers = []
for nlayers in range(1, 5):
    # Building Model

    nconv1 = 192
    nconv2 = 192
    model, in_channels = model_builder.build_model(NN=len(ineur), n_layers=nlayers, n_conv=nconv1, n_conv_mid=nconv2)
    model_name = model_builder.create_model_name(data.mouse_names[mouse_id], data.exp_date[mouse_id], n_layers=nlayers, in_channels=in_channels, seed=seed)
    
    model_path = os.path.join(weight_path, model_name)
    print('model path: ', model_path)
    model = model.to(device)


    # Training the model
    print(device)
    if not os.path.exists(model_path):
        best_state_dict = model_trainer.train(model, spks_train, spks_val, img_train, img_val, device=device)
        torch.save(best_state_dict, model_path)
        print('saved model', model_path)
    model.load_state_dict(torch.load(model_path))
    print('loaded model', model_path)

    # test model
    test_pred = model_trainer.test_epoch(model, img_test)
    print('test_pred: ', test_pred.shape, test_pred.min(), test_pred.max())


    test_fev, test_feve = metrics.feve(spks_rep_all, test_pred)
    print('FEVE (test, all): ', np.mean(test_feve))

    threshold = 0.15
    print(f'filtering neurons with FEV > {threshold}')
    valid_idxes = np.where(test_fev > threshold)[0]
    print(f'valid neurons: {len(valid_idxes)} / {len(test_fev)}')
    print(f'FEVE (test, FEV>0.15): {np.mean(test_feve[test_fev > threshold])}')

core shape:  torch.Size([1, 192, 33, 65])
input shape of readout:  (192, 33, 65)
model name:  FX8_051623_1layer_192_clamp_norm_depthsep_pool.pt
model path:  ./checkpoints_trained/FX8_051623_1layer_192_clamp_norm_depthsep_pool.pt
cuda
Learning rate = 0.001
epoch 0, train_loss = 0.8533, val_loss = 0.8408, varexp_val = 0.0102, time 41.82s
epoch 1, train_loss = 0.8374, val_loss = 0.8358, varexp_val = 0.0166, time 79.99s
epoch 2, train_loss = 0.8322, val_loss = 0.8309, varexp_val = 0.0235, time 118.18s
epoch 3, train_loss = 0.8265, val_loss = 0.8252, varexp_val = 0.0310, time 156.33s
epoch 4, train_loss = 0.8206, val_loss = 0.8208, varexp_val = 0.0369, time 194.50s
epoch 5, train_loss = 0.8168, val_loss = 0.8160, varexp_val = 0.0431, time 232.67s
epoch 6, train_loss = 0.8144, val_loss = 0.8150, varexp_val = 0.0444, time 270.83s
epoch 7, train_loss = 0.8127, val_loss = 0.8150, varexp_val = 0.0448, time 308.98s
epoch 8, train_loss = 0.8113, val_loss = 0.8121, varexp_val = 0.0486, time 347.15s

In [9]:
from minimodel import metrics
test_fev, test_feve = metrics.feve(spks_rep_all, test_pred)

threshold = 0.15
print(f'filtering neurons with FEV > {threshold}')
valid_idxes = np.where(test_fev > threshold)[0]
print(f'valid neurons: {len(valid_idxes)} / {len(test_fev)}')
print(f'FEVE (test): {np.mean(test_feve[test_fev > threshold])}')

filtering neurons with FEV > 0.15
valid neurons: 2217 / 5804
FEVE (test): 0.7174394130706787
